In [1]:
# import streamlit as st
# streamlit is for later use on web app, ignore now

import yfinance as yf
import numpy as np
from scipy.stats import norm
from datetime import datetime
import pandas as pd

# Workflow

1. User input ticker 
- TODO: company name should also works
2. Provide valid expiration date to choose
3. Provide any specific threshold, such as Greeks, IV or Strike Price Bound


In [6]:


# 1. Function to retrieve available expiration dates for a ticker
def get_available_expirations(ticker):
    """
    Retrieve available expiration dates for a given ticker using yfinance.
    """
    stock = yf.Ticker(ticker)
    available_expirations = stock.options  # Get list of expiration dates from yfinance
    return available_expirations

# 2. Black-Scholes model functions to calculate Greeks for both Call and Put options
def black_scholes_greeks(S, K, T, r, sigma, option_type='call'):
    """
    Calculate the Black-Scholes Greeks (Delta, Gamma, Theta, Vega, Rho) for call or put options.
    
    Parameters:
    S : float : Current stock price
    K : float : Option strike price
    T : float : Time to expiration (in years)
    r : float : Risk-free interest rate
    sigma : float : Implied volatility
    option_type : str : 'call' or 'put'
    
    Returns:
    delta, gamma, theta, vega, rho
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        delta = norm.cdf(d1)
        rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    elif option_type == 'put':
        delta = norm.cdf(d1) - 1
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100

    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2 if option_type == 'call' else -d2)) / 365
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    
    return delta, gamma, theta, vega, rho



In [3]:

# Function to get stock price
def get_stock_price(ticker):
    """
    Fetches the current stock price for a given ticker using yfinance.
    """
    ticker_data = yf.Ticker(ticker).history(period='1d')
    if ticker_data.empty:
        raise ValueError(f"No historical data found for {ticker}.")
    return ticker_data['Close'].iloc[0]

In [4]:
# Function to filter options based on Greek thresholds and strike price range
def filter_options_by_greeks(ticker, expiration_dates, strike_threshold=None, greek_thresholds=None, option_type=None, r=0.01):
    """
    Parameters:
    ticker : str : Stock ticker symbol
    expiration_dates : list : List of expiration dates to check
    strike_threshold : float : Threshold to filter options with strike prices within current price +/- threshold
    greek_thresholds : dict : thresholds for each Greek (e.g., {'delta': ('>', 0.5)})
    option_type : str : 'call', 'put', or None (for both)
    r : float : Risk-free interest rate
    
    Returns:
    result_df : DataFrame : Filtered options based on the user's criteria.
    """
    result_df = pd.DataFrame()
    current_price = get_stock_price(ticker)

    for expiration_date in expiration_dates:
        stock = yf.Ticker(ticker)
        options = stock.option_chain(expiration_date)

        # Filter based on user choice for calls, puts, or both
        option_types = []
        if option_type == 'call':
            option_types.append(options.calls)
        elif option_type == 'put':
            option_types.append(options.puts)
        else:
            option_types = [options.calls, options.puts]
        
        filtered_options = []
        for option_data in option_types:
            for _, row in option_data.iterrows():
                strike_price = row['strike']
                last_price = row['lastPrice']
                implied_volatility = row['impliedVolatility']
                T = (datetime.strptime(expiration_date, '%Y-%m-%d') - datetime.now()).days / 365  # Time to expiration in years
                
                # Check if the strike price is within the +/- threshold of the current price
                if strike_threshold and not (current_price - strike_threshold <= strike_price <= current_price + strike_threshold):
                    continue  # Skip this option if it's outside the strike price range

                # Calculate Greeks using Black-Scholes model
                delta, gamma, theta, vega, rho = black_scholes_greeks(current_price, strike_price, T, r, implied_volatility, 'call' if 'call' in row['contractSymbol'].lower() else 'put')
                
                add_option = True
                greek_values = {'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}
                
                if greek_thresholds:
                    for greek, (operator, value) in greek_thresholds.items():
                        if operator == '>':
                            if not (greek_values[greek] > value):
                                add_option = False
                                break
                        elif operator == '<':
                            if not (greek_values[greek] < value):
                                add_option = False
                                break
                
                if add_option:
                    filtered_options.append({
                        'Type': 'Call' if 'call' in row['contractSymbol'].lower() else 'Put',
                        'Strike': strike_price,
                        'Last Price': last_price,
                        'Delta': delta,
                        'Gamma': gamma,
                        'Theta': theta,
                        'Vega': vega,
                        'Rho': rho
                    })

        # Append filtered results to result dataframe
        if len(filtered_options) > 0:
            df = pd.DataFrame(filtered_options)
            df['Expiration Date'] = expiration_date
            result_df = pd.concat([result_df, df], ignore_index=True)

    return result_df


In [7]:


# Example showing how a user might use these functions
def user_input():
    # Input for ticker symbol
    ticker = input("Enter the ticker symbol: ").upper()  # Example user input for ticker
    
    # Fetch available expiration dates
    available_expirations = get_available_expirations(ticker)
    print("Available expiration dates: ", available_expirations)
    
    # Input for expiration dates
    expiration_dates = input("Enter expiration date(s) (comma-separated, in YYYY-MM-DD format): ").split(",")
    
    # Input for option type (call, put, or both)
    option_type = input("Enter 'call', 'put', or leave blank for both: ").lower() or None
    
    # Input for strike price threshold
    strike_threshold = float(input("Enter the strike price +/- threshold (e.g., 10): ").strip() or 0)
    
    # Input for Greek thresholds
    greek_thresholds = {}
    greek_input = input("Would you like to filter by Greeks? (yes/no): ").lower()
    if greek_input == 'yes':
        for greek in ['delta', 'gamma', 'theta', 'vega', 'rho']:
            threshold = input(f"Enter {greek} threshold (e.g., >0.5, <0.2) or leave blank to ignore: ").strip()
            if threshold:
                operator = threshold[0]
                value = float(threshold[1:])
                greek_thresholds[greek] = (operator, value)

    # Filter options using the user-provided inputs
    filtered_df = filter_options_by_greeks(ticker, expiration_dates, strike_threshold, greek_thresholds, option_type)
    
    # Show filtered results
    if filtered_df.empty:
        print(f"No options match your criteria for {ticker}.")
    else:
        print(f"Filtered options for {ticker}:")
        print(filtered_df)

# Run example
user_input()

Available expiration dates:  ('2024-10-18', '2024-10-25', '2024-11-01', '2024-11-08', '2024-11-15', '2024-11-22', '2024-11-29', '2024-12-20', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-05-16', '2025-06-20', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-06-18', '2026-09-18', '2026-12-18', '2027-01-15')
Filtered options for NVDA:
   Type  Strike  Last Price     Delta     Gamma     Theta      Vega       Rho  \
0   Put   122.0       12.68 -0.276966  0.017263 -0.142290  0.110615 -0.025226   
1   Put   123.0       11.95 -0.294908  0.017905 -0.145672  0.113966 -0.026899   
2   Put   124.0       11.30 -0.304714  0.019608 -0.137117  0.115666 -0.027624   
3   Put   125.0       11.32 -0.329930  0.019530 -0.147274  0.119625 -0.030101   
4   Put   126.0       10.30 -0.351923  0.019733 -0.153148  0.122604 -0.032234   
5   Put   127.0        9.91 -0.372212  0.020244 -0.155165  0.124970 -0.034151   
6   Put   128.0        9.28 -0.392653  0.020832 -0.155778  0.126990 